# Create a Simple Reflex-Based Lunar Lander Agent

In this example, we will use Gymnasium, an environment to train agents via reinforcement learning (RL). We will not use RL here but just use the environment with a custom simple reflex-based agent. 

## Install Gymnasium

The documentation for Gymnasium is available at https://gymnasium.farama.org/ 

Steps:
1. Create a new folder and open it with VS Code and install all needed Python Extensions in VS Code.
2. Create a new virtual environment (CTRL-Shift P Python Create Environment...)
3. I needed to install swig and the Python C++ headers on WSL2 via the terminal
    * `sudo apt install swig`
    * `sudo apt-get install python3-dev` 
4. Install gymnasium with the needed extras

In [1]:
%pip install -q swig
%pip install -q gymnasium[box2d,classic_control]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## The Lunar Lander Environment 

![Luna Lander image](https://gymnasium.farama.org/_images/lunar_lander.gif)

The documentation of the environment is available at: https://gymnasium.farama.org/environments/box2d/lunar_lander/

* Performance Measure: A reward of -100 or +100 points for crashing or landing safely respectively. We do not use 
  intermediate rewards here.

* Environment: This environment is a classic rocket trajectory optimization problem. A ship needs to land safely. The space is **continuous** with
  x and y coordinates in the range [-2.5, 2.5]. The landing pad is at coordinate (0,0).

* Actuators:  According to Pontryagin’s
  maximum principle, it is optimal to fire the engine at full throttle or turn it off. This is the reason why this environment has discrete actions: engine on or off. There are four discrete actions available:

    - 0: do nothing
    - 1: fire left orientation engine
    - 2: fire main engine
    - 3: fire right orientation engine

* Sensors: Each observation is an 8-dimensional vector: the coordinates of the lander in x & y, its linear velocities in x & y, its angle, its angular velocity, and two booleans that represent whether each leg is in contact with the ground or not.

Gymnasim environments are implemented as classes with a `make` method to create the environment, a `reset` method, and a `step` method to execute an action.
To use it with an agent function that expects percetps and returns an action, we need write glue code that connects the environment with the agent function.

In [2]:
import gymnasium as gym

def run_episode(agent_function, max_steps=1000):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialize the environment
    env = gym.make("LunarLander-v3", render_mode="human")

    # Reset the environment to generate the first observation (use seed=42 in reset to get reproducible results)
    observation, info = env.reset()

    # run one episode
    for _ in range(max_steps):
        # call the agent function to select an action
        action = agent_function(observation)

        print (f"Obs: {observation} -> Action: {action}")

        # step: execute an action in the environment
        observation, reward, terminated, truncated, info = env.step(action)
    
        env.render()

        if terminated:
            print(f"Final Reward: {reward}")
            break
    
    env.close()
    return reward

Note: `env.render()` shows the environment when the notebook is locally run (e.g., in VScode). On Colab, you cannot see the environment because the code is run on a headless server (i.e., a server without a display). There are some workarounds you can google.

## Example: A Random Agent

We ranomly return one of the actions. The environment accepts the integers 0-3.


In [3]:
import numpy as np

def random_agent_function(observation): 
    """A random agent that selects actions uniformly at random. It ignores the observation."""
    return np.random.choice([0, 1, 2, 3], p=[0.25, 0.25, 0.25, 0.25])

run_episode(random_agent_function)

Obs: [-0.00380688  1.4146793  -0.38559765  0.16706777  0.00441789  0.08734368
  0.          0.        ] -> Action: 3
Obs: [-0.00754147  1.4178544  -0.37597352  0.14110895  0.00690886  0.04982443
  0.          0.        ] -> Action: 1
Obs: [-0.01137257  1.4204264  -0.38807827  0.11427848  0.01182475  0.09832673
  0.          0.        ] -> Action: 3
Obs: [-0.01511049  1.42241    -0.3763818   0.08813827  0.01438774  0.05126483
  0.          0.        ] -> Action: 3
Obs: [-0.01875496  1.4237846  -0.3646623   0.06109359  0.01460283  0.0043018
  0.          0.        ] -> Action: 1
Obs: [-0.02246113  1.4245554  -0.37241158  0.03423471  0.01637302  0.03540721
  0.          0.        ] -> Action: 3
Obs: [-0.02609034  1.4247192  -0.3627539   0.00727727  0.01620632 -0.00333434
  0.          0.        ] -> Action: 3
Obs: [-0.02963142  1.4242845  -0.35170573 -0.01929547  0.01382565 -0.04761794
  0.          0.        ] -> Action: 0
Obs: [-0.03317251  1.4232497  -0.3516982  -0.04596416  0.01144588

-100

## A Simple Reflex-Based Agent

To make the code easier to read, we use enumerations for actions (integers) and observations (index in the observation vector).

In [4]:
from enum import Enum

class Act(Enum):
    LEFT = 1
    RIGHT = 3
    MAIN = 2
    NO_OP = 0

class Obs(Enum):
    X = 0
    Y = 1
    VX = 2
    VY = 3
    ANGLE = 4
    ANGULAR_VELOCITY = 5
    LEFT_LEG_CONTACT = 6
    RIGHT_LEG_CONTACT = 7


In [5]:
def rocket_agent_function(observation):
    """A simple agent function."""

    # run the main thruster, if the lander is falling too fast
    if observation[Obs.VY.value] < -.3:  
        return Act.MAIN.value

    return Act.NO_OP.value 

run_episode(rocket_agent_function)

Obs: [-0.00431089  1.420157   -0.43665448  0.41051662  0.00500196  0.09890874
  0.          0.        ] -> Action: 0
Obs: [-0.00862207  1.4288156  -0.43606344  0.38480332  0.00988956  0.09776135
  0.          0.        ] -> Action: 0
Obs: [-0.01293354  1.4368756  -0.43607894  0.35818112  0.0147742   0.09770175
  0.          0.        ] -> Action: 0
Obs: [-0.0172451   1.4443358  -0.43609333  0.33150896  0.01965824  0.09768949
  0.          0.        ] -> Action: 0
Obs: [-0.02155676  1.4511964  -0.4361077   0.30483818  0.0245415   0.09767415
  0.          0.        ] -> Action: 0
Obs: [-0.0258687   1.4574572  -0.43612212  0.2781681   0.02942399  0.09765888
  0.          0.        ] -> Action: 0
Obs: [-0.03018064  1.4631181  -0.43613648  0.2514981   0.03430572  0.09764364
  0.          0.        ] -> Action: 0
Obs: [-0.03449287  1.4681795  -0.43615085  0.22482799  0.03918669  0.09762833
  0.          0.        ] -> Action: 0
Obs: [-0.0388052   1.4726412  -0.43616515  0.19815774  0.0440668

-100

## Evaluating the Agent

Run the agent on 100 problems and report the average reward.

In [6]:
import numpy as np

def run_episode_test(agent_function):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialise the environment
    env = gym.make("LunarLander-v3", render_mode=None)

    # Reset the environment to generate the first observation
    observation, info = env.reset()

    # run one episode (max. 1000 steps)
    for _ in range(1000):
        # call the agent to select an action
        action = agent_function(observation)

        # step (transition) through the environment with the action
        observation, reward, terminated, truncated, info = env.step(action)
      
        if terminated:
            break
    
    env.close()
    return reward

def run_episodes(agent_function, n=1000):
    """Run multiple episodes with the given agent and return the rewards for each episode."""
    return [run_episode_test(agent_function) for _ in range(n)]

rewards = run_episodes(rocket_agent_function)
print(rewards)

print(f"Average reward: {np.average(rewards)}")
print(f"Success rate: {np.sum(np.array(rewards) == 100)}/{len(rewards)}")

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -10

This is not great!


## Implement A Better Reflex-Based Agent

Build a better that uses its right and left thrusters to land the craft (more) safely. Test your agent function using 100 problems.

In [7]:
# Code goes here